# **GE - TD sur la détection de tumeurs dans des mammographies avec  Faster-RCNN**

# **1. Install les APIs pertinentes**

Exécutez la cellule suivante.

In [ ]:
%%bash

# Install pycocotools
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
python setup.py build_ext install
cd ..
cd ..

# Install torchvision useful functions
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0


cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

cd ..

pip install pycocotools

# **2. Import des packages Python pertinent pour le TD**

Exécutez la cellule suivante.

In [ ]:
!pip install torch==1.13.1

In [ ]:
import os
import numpy as np
import torch
import time
import torch.utils.data
import matplotlib.pyplot as plt
from PIL import Image
import imageio
from engine import train_one_epoch, evaluate
import utils
from tqdm import tqdm

# **3. La base de données MIAS**


Les exercices de ce TP utiliseront tous la base données publique appelée
 **Mammographic Image Analysis Society (MIAS) dataset**.

- Le dataset MIAS réunit **322** mammographies gauche et droite.
- La taille d'une mammographie est de 1024x1024 pixels.
- Chaque mammographie comporte plusieurs annotations (cf. le fichier Info.txt), par exemple:
`mdb010 F CIRC B 525 425 33`
                1st column: MIAS database reference number.

                2nd column: Character of background tissue:
                                F - Fatty
                                G - Fatty-glandular
                                D - Dense-glandular

                3rd column: Class of abnormality present:
                                CALC - Calcification
                                CIRC - Well-defined/circumscribed masses
                                SPIC - Spiculated masses
                                MISC - Other, ill-defined masses
                                ARCH - Architectural distortion
                                ASYM - Asymmetry
                                NORM - Normal

                4th column: Severity of abnormality:
                                B - Benign
                                M - Malignant

                5th,6th columns: x,y image-coordinates of centre of abnormality.

                7th column: Approximate radius (in pixels) of a circle enclosing the abnormality.

## 3.a. Télécharger le dataset MIAS

Exécutez la cellule suivante pour télécharger le dataset MIAS.

In [ ]:
%%bash

# Download the ddataset
mkdir mias-db && cd mias-db
wget http://peipa.essex.ac.uk/pix/mias/all-mias.tar.gz
tar -zxvf all-mias.tar.gz
rm all-mias.tar.gz && cd ..

## 3.b. Répartir le dataset entre train, validation et test sets.

#### **Vous devez trouver les coordonnées des bounding box carrées englobant les tumeurs**.

Dans le dataset MIAS une anormalité dans la mammographie est localisée par les coordonnées du pixel $(x, y)$ correspondant au centre de la lésion ainsi que le rayon $r$ en pixels du cercle englobant la lésion. En utilisant $(x, y)$ et $r$, vous pouvez en déduire les coordonnées de la bounding box carrée correspondante entourant la lésion.

- Complétez la fonction `get_square_bounding_box(file_info)` qui retourne les coordonnées de la bounding box.




In [ ]:
# Path to database
mias_db_path = './mias-db/'
info_file = 'Info.txt'

In [ ]:
def get_square_bounding_box(file_info):

    if 'NORM' in file_info:
        bbox = []
    else:
        x, y, r = int(file_info.split(' ')[4]), 1024-int(file_info.split(' ')[5]), int(file_info.split(' ')[6])
        """FILL HERE"""
        bbox = ...
    return bbox


def read_dataset_info(mias_db_path, info_file, accepted_format=None, to_exclude=[]):
    with open(os.path.join(mias_db_path, info_file), 'r') as fp:
        if accepted_format is not None:
            info = [f.strip() for f in fp.readlines() if f.startswith(accepted_format) and not f.startswith(to_exclude)]
        else:
            info = [f.strip() for f in fp.readlines() if f.startswith('mdb') and not f.startswith(to_exclude)]

    dataset_info = {}
    for file_info in info:
        img_path = os.path.join(mias_db_path, file_info.split(' ')[0] + '.pgm')
        class_name = "NOTUMOUR" if 'NORM' in file_info.split(' ')[2] else "TUMOUR"
        bbox = get_square_bounding_box(file_info)

        dataset_info[img_path] = {
                                   "class_name": class_name,
                                   "bbox": bbox}
    return dataset_info

In [ ]:
# Some cases contain abnormalities so we will exclude them.
to_exclude = ('mdb216', 'mdb233', 'mdb245', 'mdb059')
# Images to include in the validation set
val_set = ('mdb001', 'mdb002', 'mdb005', 'mdb010', 'mdb012', 'mdb013')
# Images to include in the test set
test_set = ('mdb090', 'mdb091', 'mdb121', 'mdb134', 'mdb145', 'mdb218')

In [ ]:
train_dataset = read_dataset_info(mias_db_path, info_file, accepted_format=None, to_exclude=to_exclude + val_set + test_set)
validation_dataset = read_dataset_info(mias_db_path, info_file, accepted_format=val_set, to_exclude=to_exclude)
test_dataset = read_dataset_info(mias_db_path, info_file, accepted_format=test_set, to_exclude=to_exclude)

In [ ]:
for img_path in test_dataset:

    image = imageio.imread(img_path)
    class_name = test_dataset[img_path]['class_name']
    x1, y1, x2, y2  = test_dataset[img_path]["bbox"]

    plt.imshow(image, cmap="gray")
    plt.plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], 'b-')
    plt.axis('off')
    plt.title(class_name)
    plt.show()

## 3.c. Création de la classe `Dataset`.

Pour créer un dataset vous allez devoir utiliser la classe Pytorch  `Dataset`. La structure de cette classe ressemble à ce qui suit:

        class MyDataset(torch.utils.data.Dataset):

            def __init__(self, args):
                ...

            def __getitem__(self):
                return pil_image, target

            def __len(self)__:
                return len(self.img_list)


**i. Pour créer votre propre classe `Dataset`, vous allez override plusieurs méthodes**
- le constructeur `__init__(self, args, transforms)`: Un détail important de cette méthode est qu'elle doit avoir prendre en entrée la variable `transforms`. Celle-ci sera utilisée par la méthode  `__getitem__(self, image_id)` pour faire de l'augmentation de données.
- la méthode `__getitem__(self, image_id)`. Cette fonction doit retourner une image de type PIL, et un dictionnaire `target` qui contient les informations "vérité terrain" de l'image. Par exemple, le label de classification, les coordonnées de la bounding box... Les transformations liées à l'augmentation de données ont lieu dans cette fonction grâce à la varibale `transforms`.
- la méthode `__len__(self)` qui retourne la taille du dataset.


Dans votre classe, vous devez aussi coder:

**ii. Dans le constructeur:**
- `self.img_dict`: un dictionnaire qui associe l'indice `idx` d'une image aux informations la concernant (chemin vers l'image, coordonnées de la bounding box, label de classification). __Attention__: que se passe-t-il quand il n'y a pas de lésion dans la mammographie ? Il est nécessaire de créer de fausses bounding boxes.

        self.img_dict[idx] = {'path': ...,
                              'bbox': [...],
                              'class': ...}
                             
- `self.class_names`: un dictionnaire qui associe le nom `string`des classes aux labels lus par le modèle.

        self.class_names = {"TUMOUR": 1,
                            "NOTUMOUR": 2}
                            
- `self.idx2class`: un dictionnaire qui fait l'inverse de `self.class_names`.

**iii. Nouvelles méthodes:**
- `add_random_bbox(self)`: cette fonction parcourt le dataset et créer des bounding box de façon aléatoire pour les mammographies ne présentant pas de lésion. Cela permerttra de donner des exemples de tissus sains au modèle pendant l'entraînement.
- `load_image(self, img_idx)`: cette fonction retourne une image RGB de type PIL dont l'indice sera `img_idx`.



In [ ]:
class CancerDataset(torch.utils.data.Dataset):
    def __init__(self, split_dataset, transforms=None, name="train"):
        self.split_dataset = split_dataset
        self.name = name
        self.transforms = transforms

        self.class_names = {
            "TUMOUR": 1,
            "NOTUMOUR": 2,
        }

        self.add_random_bbox()
        self.idx2class = {self.class_names[name]: name for name in self.class_names}
        self.img_dict = {img_idx:{"path":  img_path,
                                  "class": self.class_names[self.split_dataset[img_path]['class_name']],
                                  "bbox":  self.split_dataset[img_path]['bbox']}
                          for img_idx, img_path in enumerate(self.split_dataset)}

    def __getitem__(self, img_idx):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly.
        """
        """FILL HERE
        Create the image, label and bbox variables using the implemented img_dict dictionary.
        """
        image = ...
        class_name = ...
        bbox = ...

        # Compute the area of the bounding box
        x1, y1, x2, y2 = bbox
        area = (x2 - x1) * (y2 - y1)

        # Convert everything to tensor
        img_idx = torch.tensor([img_idx])
        bbox = torch.as_tensor([bbox], dtype=torch.float32)
        class_name = torch.as_tensor([class_name], dtype=torch.int64)
        area = torch.as_tensor([area], dtype=torch.float32)

        # Compute the area of the bounding box
#         area = (bbox[:, 3] - bbox[:, 1]) * (bbox[:, 2] - bbox[:, 0])
#         area = torch.as_tensor(area)

        # Use the COCO template for targets to be able to evaluate the model with COCO API
        target = {"boxes": bbox,
                  "labels": class_name,
                  "image_id": img_idx,
                  "area": area,
                  "iscrowd": torch.as_tensor([0], dtype=torch.int64)}

        # Important line! don't forget to add this
        if self.transforms:
            image, target = self.transforms(image, target)
        # return the image, the boxlist and the idx in your dataset
        return image, target

    def __len__(self):
        return len(self.split_dataset)



    def add_random_bbox(self):
        # Add a random bounding box for all images that are NORMAL.
        for img_path in self.split_dataset:
            if self.split_dataset[img_path]['class_name'] == "NOTUMOUR":

                # where is there something in the mammogram ?
                img = imageio.imread(img_path)

                # Define random radius that should not be bigger than image
                radius = np.random.randint(10, 70)

                # Set borders at zero to avoid having a bounding box that is outside the mammogram
                new_img = np.zeros(img.shape)
                new_img[radius:-radius, radius:-radius] = img[radius:-radius, radius:-radius]
                mask = new_img > 50
                mask_id_x, mask_id_y = np.where(mask)

                random_id_x, random_id_y = np.random.randint(len(mask_id_x)), np.random.randint(len(mask_id_y))
                center_x, center_y = mask_id_x[random_id_x], mask_id_y[random_id_y]
                bbox = [center_x-radius, center_y-radius, center_x+radius, center_y+radius]
                self.split_dataset[img_path]['bbox'] = bbox



    def load_image(self, img_idx):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly.
        """
        img_path = self.img_dict[img_idx]['path']
        image = imageio.imread(img_path)[..., np.newaxis]
        image = np.concatenate((image, image, image), axis=2)
        image = Image.fromarray(image).convert("RGB")
        return image


## 3.d. Data augmentation

La fonction `get_transform(train)` retourne les images aléatoirement pendant l'entraînement du modèle.

Afin de renverser une image et d'appliquer la même transformation aux coordonnées de la bounding box, il faut coder une classe `RandomHorizontalFlip(object)`, dont la structure est montrée ci-dessous.

class RandomHorizontalFlip(object):

        def __init__(self, prob):
            self.prob = prob

        def __call__(self, image, target):
            if random.random() < self.prob:
                height, width = image.shape[-2:]
                # Flip image
                image = image.flip(-1)
                # Flip bounding box coordinates
                bbox = target["boxes"]
                bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
                target["boxes"] = bbox
            return image, target

Vous pouvez coder vos propre fonction d'augmentaiton de données si vous le souhaitez.

    

In [ ]:
import transforms as T
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## 3.e. Création des train, validation et test sets

- Créez les train, validation and test sets en utilisant la classe `CancerDataset`.
- A quel point le training set est-il déséquilibré ?

In [ ]:
""" FILL HERE
create your train val and test datasets
"""
train = ...
val = ...
test = ...

print("Number of images in training set: {}".format(len(train)))
print("Number of images in validation set: {}".format(len(val)))
print("Number of images in test set: {}".format(len(test)))

In [ ]:
# How imbalanced is the training set ?
class_counts = {"TUMOUR": 0, "NOTUMOUR": 0}
for img_idx in train.img_dict:
    class_name = train.idx2class[train.img_dict[img_idx]['class']]
    if "NO" in class_name: class_counts['NOTUMOUR'] += 1
    else: class_counts['TUMOUR'] += 1

print(class_counts)

## 3.f. Visualisez quelques images et leurs bounding boxes

In [ ]:
for img_idx in range(20, 25):

    img_path = train.img_dict[img_idx]['path']
    image = imageio.imread(img_path)
    class_name = train.idx2class[train.img_dict[img_idx]['class']]
    x1, y1, x2, y2  = train.img_dict[img_idx]["bbox"]

    plt.imshow(image, cmap="gray")
    if "NOTUMOUR" in class_name:
        plt.plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], 'g-')
    else:
        plt.plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], 'r-')
    plt.axis('off')
    plt.title(class_name)
    plt.show()

## 3.g. Create the data loaders


In this section, we instantiate **data loaders** that will be used to generate batches of images on the fly during training.

For each of the created datasets, you need to call `torch.utils.data.DataLoader` and define :
- the `batch_size`,
- whether to randomly shuffle the dataset so the dataloaders will return random samples by setting the `shuffle` parameter to `True` or `False`. Typically, you want to shuffle your training dataset. It does not matter for the validation and test sets.
- the number of processes that should be used to load each batch using `num_workers`.

In [ ]:
# Data loaders
# torch.manual_seed(1)

train_data_loader = torch.utils.data.DataLoader(
    train, batch_size=8, shuffle=True, num_workers=1,
    collate_fn=utils.collate_fn)

val_data_loader = torch.utils.data.DataLoader(
    val, batch_size=1, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test, batch_size=1, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)

# **4. Faster Region-based Convolutional Network model - [code](https://github.com/pytorch/vision/blob/master/torchvision/models/detection/faster_rcnn.py), [article](https://arxiv.org/abs/1506.01497)**

L'architecture du modèle **FasterRCNN** comprend le RPN comme algorithme proposant des régions d'intérêt et le modèle Fast RCNN comme détecteur.

Dans cet exercice nous entraînerons un **Faster R-CNN model avec un ResNet-50-FPN backbone** à la place de VGG.

![](https://bit.ly/3BoJCjj)

>  L'entrée du modèle doit être une liste de tensors (un par image), chacun de dimensions `[C, H, W]`  (nombre de canaux de couleur d'abord) et dont les valeurs devraient être comprise environ entre à et 1.  Les images peuvent ne pas avoir les mêmes dimensions.

> Pendant l'**entraînement**, le modèle prend la liste de tensors et les dictionnaires `targets` correspondants i.e. une liste de dictionnaires. Chaque dictionnaire `target` contients:
> - `boxes` (`FloatTensor` de dimmensions `[N, 4]`): les coordonnées des bounding boxes de référents sous format [x1, y1, x2, y2], avec `0 <= x1 < x2 <= W` et `0 <= y1 < y2 <= H.`
> - `labels` (`Int64Tensor` de dimmension `[N]`): le label de classification pour chaque bounding box de référence.
>- `image_id` (`Int64Tensor` de dimmension `[1]`): l'indice identifiant l'image. Il doit être unique pour chaque image contenue dans le dataset considéré
>- `area` (`Tensor` de dimmension `[N]`): l'aire des bounding boxes. Elle est utilisée pendant l'évaluation, pour séparer les résultats des métriques en fonction de la taille des bounding boxes.

> Le modèle retourne un `Dict[Tensor]` pendant l'entraînement, qui contient le label estimé de classification et les pertes de régression pour les coordonnées des bounding boxes pour le RPN et pour le RCNN.


## Exhaustive list of Faster RCNN's input arguments

**About the architecture:**
> - `backbone (nn.Module)`: the network used to compute the features for the model.
    It should contain a out_channels attribute, which indicates the number of output
    channels that each feature map has (and it should be the same for all feature maps).
    The backbone should return a single `Tensor` or an `OrderedDict[Tensor]`.
    
**About the input data:**
> Classes:
> - `num_classes (int)`: number of output classes of the model (including the background).
    If box_predictor is specified, `num_classes` should be None.

> Image size rescaling:
> - `min_size (int)`: minimum size of the image to be rescaled before feeding it to the backbone
> - `max_size (int)`: maximum size of the image to be rescaled before feeding it to the backbone

> Image normalization:
> - `image_mean (Tuple[float, float, float])`: mean values used for input normalization.
    They are generally the mean values of the dataset on which the backbone has been trained
    on
> - `image_std (Tuple[float, float, float])`: std values used for input normalization.
    They are generally the std values of the dataset on which the backbone has been trained on
    
**About the Region Proposal Network (RPN):**

> Architecture:
>- `rpn_anchor_generator (AnchorGenerator)`: module that generates the anchors for a set of feature
    maps.
>- `rpn_head (nn.Module)`: module that computes the objectness and regression deltas from the RPN

> NMS parameters:
> - `rpn_pre_nms_top_n_train (int)`: number of proposals to keep before applying NMS during training
> - `rpn_pre_nms_top_n_test (int)`: number of proposals to keep before applying NMS during testing
> - `rpn_post_nms_top_n_train (int)`: number of proposals to keep after applying NMS during training
> - `rpn_post_nms_top_n_test (int)`: number of proposals to keep after applying NMS during testing
> - `rpn_nms_thresh (float)`: NMS threshold used for postprocessing the RPN proposals

> IoU thresholds:
>- `rpn_fg_iou_thresh (float)`: minimum IoU between the anchor and the GT box so that they can be
    considered as positive during training of the RPN.
>- `rpn_bg_iou_thresh (float)`: maximum IoU between the anchor and the GT box so that they can be
    considered as negative during training of the RPN.

> RPN parameters for training:
>- `rpn_batch_size_per_image (int)`: number of anchors that are sampled during training of the RPN
    for computing the loss
>- `rpn_positive_fraction (float)`: proportion of positive anchors in a mini-batch during training
    of the RPN

> RPN parameter for inference:
> - `rpn_score_thresh (float)`: during inference, only return proposals with a classification score
    greater than `rpn_score_thresh`
    
**About bounding box processing and proposals:**

> Architecture:
>- `box_roi_pool (MultiScaleRoIAlign)`: the module which crops and resizes the feature maps in
    the locations indicated by the bounding boxes
>- `box_head (nn.Module)`: module that takes the cropped feature maps as input
>- `box_predictor (nn.Module)`: module that takes the output of box_head and returns the
    classification logits and box regression deltas.

> Inference:
>- `box_score_thresh (float)`: during inference, only return proposals with a classification score
    greater than `box_score_thresh`
> - `box_nms_thresh (float)`: NMS threshold for the prediction head. Used during inference
> - `box_detections_per_img (int)`: maximum number of detections per image, for all classes.

> Training:
>- `box_fg_iou_thresh (float)`: minimum IoU between the proposals and the GT box so that they can be
    considered as positive during training of the classification head
>- `box_bg_iou_thresh (float)`: maximum IoU between the proposals and the GT box so that they can be
    considered as negative during training of the classification head
>- `box_batch_size_per_image (int)`: number of proposals that are sampled during training of the
    classification head
>- `box_positive_fraction (float)`: proportion of positive proposals in a mini-batch during training
    of the classification head
>- `bbox_reg_weights (Tuple[float, float, float, float])`: weights for the encoding/decoding of the
    bounding boxes



## 4.a. Normalisation des données


Calculez la valeur moyenne `image_mean` et l'écart type `image_std` du training set.

Les sorties des fonctions devra être un **tuple**, un élément (moyenne ou écart type) par canal couleur.


In [ ]:
def compute_means(dataset):
    """ FILL HERE """
    return ...

In [ ]:
def compute_stds(dataset):
    """ FILL HERE """
    return ...

In [ ]:
image_mean = compute_means(train)
image_std = compute_stds(train)


print("Means: {}".format(image_mean))
print("Stds: {}".format(image_std))

## 4.b. Création du modèle.

Pour créer le modèle, vous devrez:
- appeler `torchvision.models.detection.fasterrcnn_resnet50_fpn` et lui donnér la moyenne et l'écart type du training set.
- indiquer le nombre de classes de notre problème:  `NOTUMOUR`, `TUMOUR` et une classe pour le fond de l'image donc 3 classes.
- remplacer la tête du prédicteur de bounding box par FastRCNN `FastRCNNPredictor`

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True,
                                                             image_mean=image_mean,
                                                             image_std=image_std)

# replace the classifier with a new one, that has num_classes which is user-defined
num_classes = 3  # intumomur + no tumour + background

# get number of input channels for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

## 4.c. Calculez le nombre de paramêtres du modèle

In [ ]:
""" FILL HERE """
model_parameters = ...
params = ...
print("Number of trainable parameters: {:.4e}".format(params))

# **5. Training**



## 5.a. Optimiseur et Hyperparamètres

> Définissez l'optimiseur et les hyperparamètres associés à utiliser pendant l'entraînement:
>- learning rate initial,
>- momentum,
>- weight decay,
>- ...

> utilisez un learning rate scheduler pour réduire le learning rate progressivement pendant le training.

In [ ]:
# gpu_number = 3

# torch.cuda.set_device(gpu_number)
# move model to the right device
model.cuda()
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

""" FILL HERE"""
optimizer = ...

# and a learning rate scheduler which decreases the learning rate
# Change the scheduler type if you wish
lr_scheduler = ...

## 5.b. Fonction de training et validation pour une époque.

>  Implementez votre propre fonction d'entraînement pour une seule époque. La fonction devra:
>- Parcourir le dataloader et sélectionner les batches,
>- Donner la prédiction du modèle,
>- Calculez la fonction de perte (la loss),
>- Re-initialiser l'optimiseur: pour chque mini-batch, il faut explicitement annuler tous les gradients avant de commencer la rétropropagation parce que Pytorch accumule les gradients calculés précédemment. Si on ne fait pas ça on risque une erreur `Out-Of-Memory`.
>- Faire la rétropropagation  (calcule les gradients for every pour chque paramètre dans le modèle).
>- Faire les mise-à-jour des paramètres (update les poids et biais du modèle en utilisant les gradients calculés).

> Implementez une fonction similaire mais destinées au calcul de la loss sur le validation set. Pas besoin de calculer des gradients, de rétropropager ou de faire des mises-à-jour dans cette fonction.


In [ ]:
!pip3 install tensorboard

In [ ]:
import math
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
def train_one_epoch(model, optimizer, data_loader, epoch, writer):

    # Set the model in training mode: the gradients will be saved.
    model.train()

    epoch_loss = {}
    for i, values in enumerate(data_loader):
        images, targets = values

        # Create list of input images
        images = ...
        # Create list of ground-truth dicionnaries
        targets = ...

        # Feed the training samples to the model
        # The model returns loss_dict which contains the values of every loss functions
        loss_dict = ...
        # Compute the global loss by summing all loss values
        global_loss = ...
        loss_value = ...

        # Increment the epoch's loss
        for k, v in loss_dict.items():
            epoch_loss[k] = epoch_loss.get(k, []) + [v.item()]
        epoch_loss['global_loss'] = epoch_loss.get('global_loss', []) + [loss_value]

        # If your loss is a Nan or infinite, you need to stop the training because it's failing.
        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict)
            sys.exit(1)

        # Initialize optimizer
        ...
        # Backpropagation: compute gradients
        ...
        # Update the model's parameters
        ...

    # Compute the losses over the whole epoch
    for k, v in epoch_loss.items():
        epoch_loss[k] = np.mean(v)
        writer.add_scalar('Training Loss/{}'.format(k), np.mean(v), epoch)
    writer.flush()
    return epoch_loss

In [ ]:
def validate_one_epoch(model, data_loader, epoch, writer):

    validation_loss = {}
    for i, values in enumerate(data_loader):
        images, targets = values
        images = list(image.cuda() for image in images)
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        global_loss = sum(loss for loss in loss_dict.values())
        loss_value = global_loss.item()

        # Increment the epoch's loss
        for k, v in loss_dict.items():
            validation_loss[k] = validation_loss.get(k, []) + [v.item()]
        validation_loss['global_loss'] = validation_loss.get('global_loss', []) + [loss_value]

    # Compute the losses over the whole epoch
    for k, v in validation_loss.items():
        validation_loss[k] = np.mean(v)
        writer.add_scalar('Validation Loss/{}'.format(k), np.mean(v), epoch)
    writer.flush()
    return validation_loss

## 5.c. Entraînez le modèle

Définissiez le nombre d'époques `num_epochs`pendant lesquelles on entraîne le modèle.

In [ ]:
num_epochs = 20

for epoch in range(1, num_epochs+1):

    # Train for one epoch, printing every 10 iterations
    start = time.time()
    epoch_loss = train_one_epoch(model, optimizer, train_data_loader, epoch, writer)
    result = "Epoch {} [{:.1f} s] - lr: {:.3e}:".format(epoch, time.time()-start, lr_scheduler.get_last_lr()[0])
    for k, v in epoch_loss.items(): result += "\t{}: {:.6f}".format(k, v)
    print(result)

    # Compute losses over the validation set
    validation_loss = validate_one_epoch(model, val_data_loader, epoch, writer)
    result = "Validation:"
    for k, v in validation_loss.items(): result += "\t{}: {:.6f}".format(k, v)
    print(result)


    # Update the learning rate
    lr_scheduler.step()

    # Save the model if you wish. You can add a criteria before saving, for example
    # if the validation decreases.
    # save_path = "/content/drive/My Drive/mva_td/saved_models/my_model"
    # torch.save(model, save_path)

#     torch.cuda.empty_cache()

## 5.d. Visualiser les courbes d'apprentissage avec Tensorboard

Est-ce que le modèle overfit ?


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir './runs/'

from tensorboard import notebook
notebook.list() # View open TensorBoard instances
notebook.display(port=6006, height=1000)

In [ ]:
!tensorboard --logdir=runs!

# **6. Evaluation over the test set**

> Pendant la phase d'**inference**, la modèle ne prend en entrée que la liste de tensors et retourne les prédictions sour la forme d'une liste de dictionnaires `List[Dict[Tensor]]`, un dictionnaire par iamge d'entrée. Les champs d'un dictionnaire sont les suivants, N étant e nombre de détections effectuées:
> - `boxes` (`FloatTensor` de dimensions `[N, 4]`): les boxes estimées par le modèle sous format `[x1, y1, x2, y2]`, avec `0 <= x1 < x2 <= W` et `0 <= y1 < y2 <= H`.
> - `labels` (`Int64Tensor` de dimension `[N]`): les labels de classification prédit pour chaque bounding box détectée.
> - `scores` (`Tensor` de dimension `[N]`): le score de confiance du modèle pour chaque détection.




## 6.a. Visualisation des prédictions


En mode "évaluation", le modèle n'a pas accès aux `targets` de référence. Pour passer dans ce mode, il faut utiliser la syntaxe `model.eval()`.

In [ ]:
def visualize(image, target, prediction):


    gt_bbox = target['boxes'][0].cpu().numpy()
    gt_label = target['labels'].cpu().numpy()

    pred_bboxs = prediction['boxes'].cpu().detach().numpy()
    pred_labels = prediction['labels'].cpu().detach().numpy()
    pred_scores = prediction['scores'].cpu().detach().numpy()

    plt.figure(figsize=(8, 8))

    # Plot mammogram
    image = image.mul(255).permute(1, 2, 0).cpu().byte()
    plt.imshow(image)
    plt.axis('off')

    # Plot ground-truth bounding box
    x1, y1, x2, y2 = gt_bbox
    line1, = plt.plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], 'b-', label='Ground-truth')

    # Plot predicted bounding boxes
    for i in range(len(pred_labels)):
        x1, y1, x2, y2 = pred_bboxs[i]
        score = pred_scores[i]

        if pred_labels[i] == 1:
            c = 'r'
            label='Prediction: tumour'
            plt.annotate('{:.1f}'.format(100*score), (x1, y1), c=c, fontsize='medium')
        else:
            c = 'g'
            label='Prediction: no tumour'
            plt.annotate('{:.1f}'.format(100*score), (x2, y2), c=c, fontsize='medium')
        line, = plt.plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], c, label=label)

    plt.show()



In [ ]:
model.eval()
for i, values in enumerate(test_data_loader):
        images, targets = values

        # Create list of input images
        images = list(image.cuda() for image in images)
        # Create list of ground-truth dicionnaries
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        # Feed the training samples to the model
        # The model returns  the predictions
        predictions = model(images)

        visualize(images[0], targets[0], predictions[0])

## 6.b. Evaluation

Exécutez le code suivant pour calculer la métrique "mean Average Precision" sur le test set.

In [ ]:
from engine import evaluate
device = torch.device('cuda:{}'.format(gpu_number))
evaluate(model, test_data_loader, device)